![A soccer pitch for an international match.](../assets/img/soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [34]:
# Start your code here!
import pandas as pd
import plotly.express as px

from scipy.stats import shapiro, mannwhitneyu

In [35]:
def read_data(data_path):
    group = data_path.split("_")[0]
    result_df = pd.read_csv(data_path, index_col=0)
    result_df["date"] = pd.to_datetime(result_df["date"])
    
    result_df =  result_df[(result_df['tournament'] == "FIFA World Cup") & (result_df['date']> "2002-01-01")]
    result_df["goals_scored"] =  result_df["home_score"] + result_df["away_score"]
    result_df["group"] =  group
    return result_df

men =  read_data("men_results.csv")
women =  read_data("women_results.csv")
combined_results = pd.concat([men, women])
combined_results.head()

,date,home_team,away_team,home_score,away_score,tournament,goals_scored,group
25164,2002-05-31,France,Senegal,0,1,FIFA World Cup,1,men
25165,2002-06-01,Germany,Saudi Arabia,8,0,FIFA World Cup,8,men
25166,2002-06-01,Republic of Ireland,Cameroon,1,1,FIFA World Cup,2,men
25167,2002-06-01,Uruguay,Denmark,1,2,FIFA World Cup,3,men
25168,2002-06-02,Argentina,Nigeria,1,0,FIFA World Cup,1,men


In [36]:
px.histogram(combined_results, x="goals_scored", color="group", barmode="overlay")

In [37]:
test_stat, pvalue = shapiro(combined_results.loc[combined_results["group"] == "men", "goals_scored"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
test_stat, pvalue = shapiro(combined_results.loc[combined_results["group"] == "women", "goals_scored"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat = 0.9266, p-value = 0.0000
Test Stat = 0.8491, p-value = 0.0000


- Based on the results of the Shapiro-Wilk test, which produced a p-value of 0.000 for both the control and variant groups, we reject the null hypothesis of normality.

In [38]:
# Perform right-tailed Wilcoxon-Mann-Whitney
# alternative='greater' -> the null hypothesis is that the goal scores distribution underlying `women` is not stochastically greater than the distribution underlying `men`.
test_stat, p_val = mannwhitneyu(combined_results.loc[combined_results["group"] == "women", "goals_scored"],
                                 combined_results.loc[combined_results["group"] == "men", "goals_scored"],
                                 alternative='greater'
)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, p_val))

Test Stat = 43273.0000, p-value = 0.0051


In [39]:
# Determine hypothesis test result using sig. level
if p_val <= 0.01:
    result = "reject"
else:
    result = "fail to reject"

result_dict = {"p_val": p_val, "result": result}
print(result_dict)

{'p_val': 0.005106609825443641, 'result': 'reject'}
